In [1]:
from keras.layers import Conv2D, Conv2DTranspose, Flatten, Input, Dense, Reshape
from keras.datasets import mnist
from keras import Model
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import glob
%matplotlib inline

Using TensorFlow backend.


# Train Data

In [4]:
%%time 
img_size = 64

samples = glob.glob('./lfwcrop_grey/faces/*')
data = []
print('Point 1')
for s in samples:
    img = mpimg.imread(s)
    data.append(np.expand_dims(img, 0))
        
# faces = np.concatenate(faces)

Point 1
Wall time: 2min 8s


In [5]:
faces = np.concatenate(data, axis=0)

In [6]:
faces.shape

(13233, 64, 64)

In [7]:
faces = np.expand_dims(faces, -1)

In [8]:
# prepare data
faces = faces / 255.

In [9]:
faces.shape

(13233, 64, 64, 1)

#### Fit autoencoder

In [10]:
# encoder
input_ = Input((64, 64, 1)) # 64
x = Conv2D(filters=8, kernel_size=2, strides=2, activation='relu')(input_) # 32
x = Conv2D(filters=16, kernel_size=2, strides=2, activation='relu')(x) # 16
x = Conv2D(filters=32, kernel_size=2, strides=2, activation='relu')(x) # 8
x = Conv2D(filters=64, kernel_size=2, strides=2, activation='relu')(x) # 4
x = Conv2D(filters=128, kernel_size=2, strides=2, activation='relu')(x) # 2
flat = Flatten()(x)
latent = Dense(128)(flat)

# decoder
reshape = Reshape((2,2,32)) #2
conv_2t_1 = Conv2DTranspose(filters=128, kernel_size=2, strides=2, activation='relu') # 4
conv_2t_2 = Conv2DTranspose(filters=64, kernel_size=2, strides=2, activation='relu') # 8
conv_2t_3 = Conv2DTranspose(filters=32, kernel_size=2, strides=2, activation='relu') # 16
conv_2t_4 = Conv2DTranspose(filters=16, kernel_size=2, strides=2, activation='relu') # 32
conv_2t_5 = Conv2DTranspose(filters=1, kernel_size=2, strides=2, activation='sigmoid') # 64

x = reshape(latent)
x = conv_2t_1(x)
x = conv_2t_2(x)
x = conv_2t_3(x)
x = conv_2t_4(x)
decoded = conv_2t_5(x) # 64

autoencoder = Model(input_, decoded)
encoder = Model(input_, latent)

decoder_input = Input((128,))
x_ = reshape(decoder_input)
x_ = conv_2t_1(x_)
x_ = conv_2t_2(x_)
x_ = conv_2t_3(x_)
x_ = conv_2t_4(x_)
decoded_ = conv_2t_5(x_) # 64
decoder = Model(decoder_input, decoded_)

In [11]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 8)         40        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 16)        528       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 32)          2080      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 64)          8256      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 2, 128)         32896     
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
__________

In [12]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [138]:
autoencoder.fit(faces, faces, epochs=50)

Epoch 1/50
13233/13233 [==============================] - 7s 566us/step - loss: 0.6311
Epoch 2/50
13233/13233 [==============================] - 7s 538us/step - loss: 0.6311
Epoch 3/50
13233/13233 [==============================] - 7s 539us/step - loss: 0.6311
Epoch 4/50
13233/13233 [==============================] - 7s 538us/step - loss: 0.6311
Epoch 5/50
13233/13233 [==============================] - 7s 536us/step - loss: 0.6311
Epoch 6/50
13233/13233 [==============================] - 7s 535us/step - loss: 0.6311
Epoch 7/50
13233/13233 [==============================] - 7s 537us/step - loss: 0.6311
Epoch 8/50
13233/13233 [==============================] - 7s 539us/step - loss: 0.6311
Epoch 9/50
13233/13233 [==============================] - 7s 538us/step - loss: 0.6311
Epoch 10/50
13233/13233 [==============================] - 7s 536us/step - loss: 0.6311
Epoch 11/50
13233/13233 [==============================] - 7s 539us/step - loss: 0.6311
Epoch 12/50
13233/13233 [================

# Save model

In [151]:
# save weights
encoder.save_weights('encoder_weights_mri.h5')
decoder.save_weights('decoder_weights_mri.h5')

# save architecture
json_encoder = encoder.to_json()
json_decoder = decoder.to_json()

with open('encoder_mri_json.txt', 'w') as file:
    file.write(json_encoder)

with open('decoder_mri_json.txt', 'w') as file:
    file.write(json_decoder)